In [202]:
from lxml import html
import requests
from BeautifulSoup import BeautifulSoup

In [203]:
page = requests.get('https://www.nwseaportalliance.com/operations/vessels')

In [204]:
# Pull existing data
soup = BeautifulSoup(page.text)

In [205]:
vessels = soup.findAll('td', {'class':'views-field views-field-title-1'})

In [206]:
vessels[0].findAll('a',href=True)[0].contents

[u'MATSON PRODUCER               ']

In [207]:
for a in vessels[0].findAll('a', href=True):
    print a['href']    # the location of vessel data 
    print a.contents[0]    # vessel name?

/node/338542
MATSON PRODUCER               


In [208]:
vessel_list = []
href_list = []
for vessel in vessels:
    atag = vessel.findAll('a',href=True)[0]
    vessel_list.append(atag.contents[0])
    href_list.append(atag['href'])

In [216]:
# Get all the ETA data
eta = soup.findAll('div',{'class':'field field-name-field-vessel-eta field-type-datestamp field-label-hidden'})
eta_list = []
for e in eta:
    eta_list.append(e.findAll('span')[0].contents[0].strip())

In [217]:
# Get all the ETD data
etd = soup.findAll('div', {'class':'field field-name-field-vessel-etd field-type-datestamp field-label-hidden'})
etd_list = []
for e in etd:
    etd_list.append(e.findAll('span')[0].contents[0].strip())

In [218]:
# Get all the ETD data
harbor = soup.findAll('div', {'class':'field field-name-field-vessel-terminal field-type-text field-label-hidden'})
harbor_list = []
for h in harbor:
    harbor_list.append(h.contents[0].split('\n')[-1].strip())

In [220]:
# If ship doesn't already exist in our database, collect it
vessel_db = []

# for i in xrange(len(vessel_list)):
for i in [0]:
    vessel = vessel_list[i].strip()
    if vessel not in ship_db:
        href = href_list[i]
        page = requests.get('https://www.nwseaportalliance.com' + href)
        soup = BeautifulSoup(page.text)
        vessel_db.append(vessel)

In [221]:
# Get the list of class names to search
vessel_class_names = []
for field in [i['class'] for i in soup.findAll('div')[1:]]:
    if 'field' in str(field):
        vessel_class_names.append(field)

In [222]:
vessel_class_names

[u'field field-name-field-vessel-eta field-type-datestamp field-label-above',
 u'field field-name-field-vessel-etd field-type-datestamp field-label-above',
 u'field field-name-field-vessel-cargotype field-type-text field-label-above',
 u'field field-name-field-vessel-stevedore field-type-text field-label-above',
 u'field field-name-field-vessel-agent field-type-text field-label-above',
 u'field field-name-field-vessel-length field-type-text field-label-above',
 u'field field-name-field-vessel-width field-type-text field-label-above',
 u'field field-name-field-vessel-draught field-type-text field-label-above',
 u'field field-name-field-vessel-berth field-type-text field-label-above',
 u'field field-name-field-vessel-terminal field-type-text field-label-above',
 u'field field-name-field-vessel-harbor field-type-text field-label-above']

In [223]:
vessel_dict = {}
for class_name in vessel_class_names:
    field_name = class_name.split(' ')[1].split('-')[-1]
    field_val = soup.findAll('div', {'class':class_name})[0].contents[0].split('\n')[-1].strip()
    vessel_dict[field_name] = field_val

In [224]:
vessel_dict

{u'agent': u'MATSON',
 u'berth': u'B',
 u'cargotype': u'LAY BERTH',
 u'draught': u'34.09',
 u'eta': u'',
 u'etd': u'',
 u'harbor': u'TAC',
 u'length': u'720.54',
 u'stevedore': u'',
 u'terminal': u'APM Terminals',
 u'width': u'95.18'}

In [ ]:
#